##**Preparación de Texto**

**Preparar notebook**

In [ ]:
## Conectar el notebook a googledrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Importar librerias necesarias
import nltk
import pandas as pd
import numpy as np
import re
import codecs
import matplotlib.pyplot as plt
from collections import Counter
from nltk.util import ngrams
from nltk import bigrams
import spacy

In [ ]:
# Importar librerias para lematizador
!pip install contexto

from contexto.lematizacion import LematizadorSpacy, LematizadorStanza
from contexto.lematizacion import lematizar_texto
from contexto.correccion import Corrector, corregir_texto

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 136 kB 35.0 MB/s 
     |████████████████████████████████| 1.9 MB 61.0 MB/s 
     |████████████████████████████████| 23.9 MB 1.2 MB/s 
     |████████████████████████████████| 2.6 MB 73.2 MB/s 
     |████████████████████████████████| 5.6 MB 49.5 MB/s 
     |████████████████████████████████| 9.4 MB 54.4 MB/s 
     |████████████████████████████████| 2.7 MB 74.2 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 1.5 MB 58.8 MB/s 
     |████████████████████████████████| 94 kB 2.4 MB/s 
     |████████████████████████████████| 691 kB 41.7 MB/s 
     |████████████████████████████████| 1.6 MB 75.6 MB/s 
     |████████████████████████████████| 100 kB 9.6 MB/s 
     |████████████████████████████████| 295 kB 69.1 MB/s 
     |████████████████████████████████| 965 kB 65.9 MB/s 
     |██████████████████████████████

In [ ]:
## Descargar el corpus de nltk para 'tokenizer', 'stopwords'
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Importar stopwords
from nltk.corpus import stopwords
stop_words_nltk = set(stopwords.words('spanish'))

from nltk.probability import ConditionalFreqDist
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


**Cargar los Datos**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ProyectoIntegrador/Datos/export.csv', encoding='latin-1')
print(df.shape)
df.head(2)

(259762, 15)


,REPORTDATE,WONUM,DESCRIPTION,STATUS,WORKTYPE,ASSETNUM,LOCATION,REGION,ANCESTOR,SOURCESYSID,FAILURECODE,PROBLEMCODE,RESUMEN,WORKLOGID,DETALLE
0,29/05/2022 02:26:00,10016825,Atención Evento M_EPM_EN_4885195,CERRADA,CPROG,NaN,32427-4,METROPOLITANA,R05-14,MAR,POSTE,CAIDO,CIERREOT,2754950,Se revisa y se encuentra poste de 12 MTS de ji...
1,01/09/2022 02:56:24,12523772,Reparar líneas primaria subterránea fallada,CERRADA,CPROG,3206258,32427-4,METROPOLITANA,R05-14,MAR,NaN,NaN,CIERREOT,3005990,Localizar falla circuito primario sub guayabal...


In [ ]:
#df = df.sample(n = 80000, replace = True, random_state=1).reset_index(drop=True)

In [ ]:
##df.to_csv('/content/drive/MyDrive/ProyectoIntegrador/Datos/sample.csv')

In [ ]:
#df = pd.read_csv('/content/drive/MyDrive/ProyectoIntegrador/Datos/sample.csv', encoding='utf-8')
#print(df.shape)
#df.head(2)

In [ ]:
df['RESUMEN'].value_counts()

CIERREOT                                                                                                248191
MATERIALRECUPERA                                                                                          1380
EJECUTADO                                                                                                  995
INTERVENTORIA                                                                                              921
CORREGIR                                                                                                   669
                                                                                                         ...  
OT POR ITEM                                                                                                  1
ACLARACIÓN DE ACTUALIZACIÓN OT                                                                               1
SE DEVUELVE A EJECUTADO SE HABIA DEVUELTO POR QUE NO COINCIDIA EL XML Y VUEVEN Y LO ENVIAN SIN CORRE         1
E

In [ ]:
#Retirar algunos tipos de OTs que no aportan información de valor
df = df[~df['RESUMEN'].isin(['MATERIALRECUPERA','SUBESTACIONES'])]
print(df.shape)

(257747, 15)


In [ ]:
df['WORKTYPE'].value_counts()

CINME    109502
ATTER     76859
CPROG     47364
PROYE     13578
OPERA      6257
MBC        2215
MP            5
Name: WORKTYPE, dtype: int64

In [ ]:
## Seleccionar registros asociados con tipos de trabajo de interés (Trabajos correctivos)
df = df[df['WORKTYPE'].isin(['CINME', 'CPROG'])]
df.shape

(156866, 15)

In [ ]:
#Seleccionar las columnas de interés para el análisis
datos = df.loc[:, ['RESUMEN', 'WORKLOGID', 'DETALLE']]
#datos = df[['RESUMEN', 'WORKLOGID', 'DETALLE']]
datos.head()

,RESUMEN,WORKLOGID,DETALLE
0,CIERREOT,2754950,Se revisa y se encuentra poste de 12 MTS de ji...
1,CIERREOT,3005990,Localizar falla circuito primario sub guayabal...
4,CIERREOT,2634320,Se retiraron 5 ramas de Palma que estaban sobr...
5,CIERREOT,2634321,R27-07 se visita Cll57c#34-36 revisión por cal...
6,CIERREOT,2742512,Con el evento Mar 4879178 en carrera 33 por 29...


In [ ]:
#Eliminar líneas sin datos en la columna "DETALLE"
datos = datos.dropna(subset = ['DETALLE']).reset_index(drop=True)
datos.shape

(156792, 3)

**Limpieza de Texto**

Creación de funciones para la limpieza de texto

In [ ]:
#carga diccionario de sinonimos
sinonimos = pd.read_csv('/content/drive/MyDrive/ProyectoIntegrador/Datos/sinonimos.csv')

In [ ]:
def cambiar_sinonimos(path):
  for index in range(sinonimos.shape[0]):
    palabra = sinonimos.iloc[index, 0 ]
    sinonimo = sinonimos.iloc[index,1 ]
    path = re.sub(r'\b'+palabra+r'\b', sinonimo , path)
    ##path = path.replace(palabra, sinonimo)
  return path

In [ ]:
# Definición del listado de palabras vacías de nltk
w_vacias = list(stop_words_nltk)

In [ ]:
##Defición de función para remover del listado de tokens stopwords y tokens de longitud menor que "n" 
def remover_stopwords (tokens):
  n=2
  for i in range(len(tokens)):
    tokens[i] = [w for w in tokens[i] if w not in w_vacias and len(w)>n]
  return tokens

In [ ]:
# Definición de función para lematizar los tokens
lematizador = LematizadorStanza('es')
def lematizar_tokens(tokens):
  for i in range(len(tokens)):
    tokens[i] = [lematizar_texto(w, lematizador=lematizador) for w in tokens[i]]
  return tokens

In [ ]:
#corrector = Corrector(lenguaje='es')
#corrector.agregar_palabras(['aisladero', 'reconectador'])

In [ ]:
## Definición de la función para la limpieza del texto en la columna "DETALLE"
def limpiar_texto (data):
  data['procesado'] = data['DETALLE'] # Crear columna para almacenar el texto procesado
  data['procesado'] = data['procesado'].apply(lambda x: re.sub('<[^>]*>', ' ', str(x))) #Remover Tags HTML
  data['procesado'] = data['procesado'].apply(lambda x: str(x).lower()) #Normalizar texto a minúscula
  data['procesado'] = data['procesado'].apply(lambda x: str(x).translate(str.maketrans('áéíóúüñ','aeiouun'))) #Cambiar tildes y ñ
  data['procesado'] = data['procesado'].apply(lambda x: re.sub('[^A-Za-z]+', ' ', str(x))) #Retirar todos los caracteres no letras
  data['procesado']=data['procesado'].apply(lambda x: cambiar_sinonimos(x)) # aplicar la función cambiar_sinonimos
  #data['procesado']=data['procesado'].apply(lambda x: corregir_texto(x, corrector=corrector))
  data['tokens'] = data['procesado'].apply(lambda x: nltk.word_tokenize(str(x))) #Crear columna con los tokens
  data['tokens_proc'] = data['tokens'] # Crear columna para almacenar los tokens procesados
  data['tokens_proc'] = remover_stopwords(data['tokens_proc']) # Remover stopwords y tokens "cortos" del listado de tokens
  return data

Limpiar texto

In [ ]:
# Limpiar el texto en la columna "DETALLE"
limpiar_texto(datos)
#Toma mas de 45min para su ejecución

<ipython-input-18-2b3bb70269ac>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokens[i] = [w for w in tokens[i] if w not in w_vacias and len(w)>n]


,RESUMEN,WORKLOGID,DETALLE,procesado,tokens,tokens_proc
0,CIERREOT,2754950,Se revisa y se encuentra poste de 12 MTS de ji...,se revisa y se encuentra poste de metros de ji...,"[se, revisa, y, se, encuentra, poste, de, metr...","[revisa, encuentra, poste, metros, jibra, reve..."
1,CIERREOT,3005990,Localizar falla circuito primario sub guayabal...,localizar falla circuito primario subestacion ...,"[localizar, falla, circuito, primario, subesta...","[localizar, falla, circuito, primario, subesta..."
2,CIERREOT,2634320,Se retiraron 5 ramas de Palma que estaban sobr...,se retiraron arbol de palma que estaban sobre ...,"[se, retiraron, arbol, de, palma, que, estaban...","[retiraron, arbol, palma, aisladero, primario,..."
3,CIERREOT,2634321,R27-07 se visita Cll57c#34-36 revisión por cal...,r se visita cll c revision por calidad del ser...,"[r, se, visita, cll, c, revision, por, calidad...","[visita, cll, revision, calidad, servicio, ins..."
4,CIERREOT,2742512,Con el evento Mar 4879178 en carrera 33 por 29...,con el evento mar en carrera por c del r donde...,"[con, el, evento, mar, en, carrera, por, c, de...","[evento, mar, carrera, trabajador, reparan, li..."
...,...,...,...,...,...,...
156787,CIERREOT,2927475,SE NORMALIZO AISLADERO SIN MARCAR FASE S PARA ...,se normalizo aisladero sin marcar fase s para ...,"[se, normalizo, aisladero, sin, marcar, fase, ...","[normalizo, aisladero, marcar, fase, solo, tra..."
156788,CIERREOT,3157566,Se visita Cr43#32-39 Ed Metropolitan en legal...,se visita cr ed metropolitan en legalizacion e...,"[se, visita, cr, ed, metropolitan, en, legaliz...","[visita, metropolitan, legalizacion, dia, ayer..."
156789,CIERREOT,3056493,Se le prestó a pollo al personal del CLD para ...,se le presto a pollo al personal del cld para ...,"[se, le, presto, a, pollo, al, personal, del, ...","[presto, pollo, personal, cld, abrir, circuito..."
156790,CIERREOT,3055378,Se aisló el reconectador R35 41 y se abrieron ...,se aislo el reconectador r y se abrieron las c...,"[se, aislo, el, reconectador, r, y, se, abrier...","[aislo, reconectador, abrieron, cuchillas, sal..."


In [ ]:
#Lematizar tokens
#datos['tokens_proc'] = lematizar_tokens(datos['tokens_proc'])

In [ ]:
# Calculo de la longitud de las cadenas de tokens y estadísticos asociados
tokens_lenght = datos['tokens_proc'].apply(lambda x: len(x))
tokens_lenght.describe()

count    156792.000000
mean         13.917955
std          10.022890
min           0.000000
25%           7.000000
50%          11.000000
75%          18.000000
max         122.000000
Name: tokens_proc, dtype: float64

In [ ]:
## Obtener listado de tokens (ya procesados)
tokens = []
for t in datos['tokens_proc']:
    tokens.extend(t)
print('Cantidad total de tokens = ',str(len(tokens)))
## Distribución de frecuencia de los tokens. Tamaño del BoW
fdist_tokens = nltk.FreqDist(tokens)
print('Tamanno del BoW=',len(fdist_tokens))

Cantidad total de tokens =  2182224
Tamanno del BoW= 40449


Análisis del BoW

In [ ]:
df_frecuencia = pd.DataFrame([[key, fdist_tokens[key]] for key in fdist_tokens.keys()], columns=['palabra', 'frecuencia'])

In [ ]:
df_frecuencia.describe()

,frecuencia
count,40449.000000
mean,53.950011
std,940.119390
min,1.000000
25%,1.000000
50%,1.000000
75%,4.000000
max,97422.000000


In [ ]:
#Encontrar tokens (palabras) con baja frecuencia de ocurrencia - Aparecen 3 veces o menos en todo el BoW
baja_frec = df_frecuencia[df_frecuencia['frecuencia']<4]
len(baja_frec)

28872

In [ ]:
# Definición del listado de otras palabras vacías y de baja frecuencia (tokens frecuencia menor o igual a 3)
otras_vacias = pd.read_csv('/content/drive/MyDrive/ProyectoIntegrador/Datos/palabras_vacias.csv')
palabras_vacias = otras_vacias['palabras_vacias'].values.tolist() + baja_frec['palabra'].values.tolist()

In [ ]:
##Defición de función que reciba la columna del dataframe con los tokens y una lista de palabras para eliminar
def remover_palabra (tokens):
  for i in range(len(tokens)):
    tokens[i] = [w for w in tokens[i] if w not in palabras_vacias]
  return tokens

Eliminar palabras de poco valor o relevancia

In [ ]:
datos['tokens_proc'] = remover_palabra(datos['tokens_proc'])
#Toma 1:20 min para finalizar

<ipython-input-30-c123455261ea>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokens[i] = [w for w in tokens[i] if w not in palabras_vacias]


In [ ]:
datos.shape
datos.head()

,RESUMEN,WORKLOGID,DETALLE,procesado,tokens,tokens_proc
0,CIERREOT,2754950,Se revisa y se encuentra poste de 12 MTS de ji...,se revisa y se encuentra poste de metros de ji...,"[se, revisa, y, se, encuentra, poste, de, metr...","[revisa, poste, reventado, vehiculo, recoje, e..."
1,CIERREOT,3005990,Localizar falla circuito primario sub guayabal...,localizar falla circuito primario subestacion ...,"[localizar, falla, circuito, primario, subesta...","[localizar, falla, circuito, primario, subesta..."
2,CIERREOT,2634320,Se retiraron 5 ramas de Palma que estaban sobr...,se retiraron arbol de palma que estaban sobre ...,"[se, retiraron, arbol, de, palma, que, estaban...","[retiraron, arbol, palma, aisladero, primario,..."
3,CIERREOT,2634321,R27-07 se visita Cll57c#34-36 revisión por cal...,r se visita cll c revision por calidad del ser...,"[r, se, visita, cll, c, revision, por, calidad...","[cll, revision, calidad, inspecciona, evidenci..."
4,CIERREOT,2742512,Con el evento Mar 4879178 en carrera 33 por 29...,con el evento mar en carrera por c del r donde...,"[con, el, evento, mar, en, carrera, por, c, de...","[mar, carrera, trabajador, reparan, linea, pri..."


Exportar los datos procesados para ser usados en el análisis 

In [ ]:
datos.to_csv('/content/drive/MyDrive/ProyectoIntegrador/Datos/datos_preparados.csv')